In [1]:
import json

from pyboolnet.external.bnet2primes import bnet_file2primes
from pyboolnet.prime_implicants import percolate
from pyboolnet.file_exchange import primes2bnet

from boolmore.conversions import prime2rr

In [2]:
ABA_A = "../../case_study/baseline_models/ABA_GA_base_A.txt"
ABA_A_json = "../../case_study/data/ABA_2017.json"
ABA_A_data = "../../case_study/data/data_20230925.tsv"

In [3]:
base_primes = bnet_file2primes(ABA_A)

json_file = json.load(open(ABA_A_json, "r"))

extra_edges = json_file["edge_pool"]

perc_primes = percolate(base_primes, remove_constants = False, copy=True)

print(primes2bnet(perc_primes))
print(len(extra_edges))

ABH1,                           
ARP_Complex,                    
CPK23,                          
CPK6,                           
DAGK,                           
ERA1,                           
GAPC1_2,                        
GCR1,                           
GEF1_4_10,                      
GTP,                            
MRP5,                           
NAD,                            
NADPH,                          
Nitrite,                        
NtSyp121,                       
PC,                             
PtdIns4_5P2,                    
PtdInsP3,                       
PtdInsP4,                       
RCN1,                           
ROP11,                          
SCAB1,                          
SPP1,                           
Sph,                            

ABA,                            ABA

8-nitro-cGMP,                   NO & ROS & cGMP
ABI1,                           !PA & !RCARs & !ROS & pHc
ABI2,                           !RCARs & !ROS
ADPRc,            

In [4]:
def get_constants(primes):

    constants = dict()

    for node in primes:
        regulators, rr, signs = prime2rr(primes[node])

        if len(regulators) == 0:
            if len(primes[node][0]) == 1:
                constants[node] = 0
            elif len(primes[node][1]) == 1:
                constants[node] = 1

    return constants

constants = get_constants(perc_primes)

print(len(constants))

24


In [5]:
# remove extra edges that involve constants
for edge in extra_edges:
    for node in constants:
        if node in edge:
            extra_edges.remove(edge)

print(len(extra_edges))

# remove constants from the primes
percolate(perc_primes, remove_constants = True, copy=False)

12


In [6]:
def primes2sif(primes, file_name):

    fp = open(file_name, "w")

    print("interactions")
    for node in primes:
        regulators, rr, signs = prime2rr(primes[node])

        for i, reg in enumerate(regulators):
            if signs[i] == "0":
                print(str(reg)+"\t-|\t"+str(node))
                fp.write(str(reg)+"\t-|\t"+str(node)+"\n")
            elif signs[i] == "1":
                print(str(reg)+"\t->\t"+str(node))
                fp.write(str(reg)+"\t->\t"+str(node)+"\n")

    fp.close()

def add_extra_edges(extra_edges, file_name):

    fp = open(file_name, "a")

    print("\nextra edges")
    for edge in extra_edges:
        if edge[2] == "0":
            print(str(edge[0])+"\t-|\t"+str(edge[1]))
            fp.write(str(edge[0])+"\t-|\t"+str(edge[1])+"\n")
        elif edge[2] == "1":
            print(str(edge[0])+"\t->\t"+str(edge[1]))
            fp.write(str(edge[0])+"\t->\t"+str(edge[1])+"\n")

    fp.close()

primes2sif(perc_primes, "ABA_A_network.sif")

add_extra_edges(extra_edges, "ABA_A_network.sif")

interactions
NO	->	8-nitro-cGMP
ROS	->	8-nitro-cGMP
cGMP	->	8-nitro-cGMP
ABA	->	ABA
PA	-|	ABI1
RCARs	-|	ABI1
ROS	-|	ABI1
pHc	->	ABI1
RCARs	-|	ABI2
ROS	-|	ABI2
8-nitro-cGMP	->	ADPRc
AtRAC1	-|	Actin_Reorganization
QUAC1	->	AnionEM
SLAC1	->	AnionEM
SLAH3	->	AnionEM
OST1	->	AquaporinPIP2_1
ABA	-|	AtRAC1
ABI1	->	AtRAC1
InsP3	->	CIS
InsP6	->	CIS
cADPR	->	CIS
CPK3_21	->	CPK3_21
Ca2c	->	CPK3_21
Ca2c	->	Ca2_ATPase
CIS	->	Ca2c
Ca2_ATPase	-|	Ca2c
CaIM	->	Ca2c
GHR1	->	CaIM
SACC	->	CaIM
H2O_Efflux	->	Closure
Microtubule_Depolymerization	->	Closure
PLC	->	DAG
AnionEM	->	Depolarization
Ca2c	->	Depolarization
H_ATPase	-|	Depolarization
KEV	->	Depolarization
K_efflux	-|	Depolarization
ABI2	-|	GHR1
ROS	->	GHR1
S1P_PhytoS1P	->	GPA1
AnionEM	->	H2O_Efflux
AquaporinPIP2_1	->	H2O_Efflux
H2O_Efflux	->	H2O_Efflux
K_efflux	->	H2O_Efflux
Malate	-|	H2O_Efflux
RCARs	-|	HAB1
ROS	-|	HAB1
Ca2c	-|	H_ATPase
ROS	-|	H_ATPase
pHc	-|	H_ATPase
PLC	->	InsP3
InsP3	->	InsP6
Ca2c	->	KEV
Depolarization	->	KOUT
NO	-|	KOUT
ROS	-|	

In [12]:
# read csv file in data.
import csv

from matplotlib.pylab import f
from numpy import number

def comment_removal(line:str) -> bool:
    return not line.startswith("#") and not line.isspace()

def convert_exps(data_loc, constants, file_name=None):

    ID, SCORE, SOURCE, PERT, NODE, VALUE = 0, 1, 2, 3, 4, 5
        
    file = open(data_loc, "r")
    lines = filter(comment_removal, file)
    data = csv.reader(lines, delimiter="\t")

    # skip the first row
    if file_name is not None:
        fp = open(file_name, "w")
        fp.write("\t".join(next(data))+"\n")
    else:
        next(data)

    number_skipped = 0

    exps = {}
    for line in data:
        # print(line)
        
        skip = False
        
        condition = f"{line[SOURCE].split('=')[0]}:{line[SOURCE].split('=')[1]}"
        
        perturbation = {}
        for pert in line[PERT].split(","):
            if pert == "":
                continue
            node, value = pert.strip().split(" ")
            if value == "KO":
                perturbation[node] = 0
            elif value == "CA":
                perturbation[node] = 1

        for node in perturbation:
            if node in constants:
                # print(f"{node} is a constant")
                skip = True
            condition += f"\t{node}:{perturbation[node]}"

        response = f"{line[NODE]}:"
        if line[NODE] in constants:
            skip = True

        if line[VALUE] == "OFF":
            response += "0"
        elif line[VALUE] == "OFF/Some":
            response += "0.25"
        elif line[VALUE] == "Some":
            response += "0.5"
        elif line[VALUE] == "Some/ON":
            response += "0.75"
        elif line[VALUE] == "ON":
            response += "1"
        else:
            raise Exception(f"{line[VALUE]} is not a valid value")
        
        if skip:
            # print(f"{line[ID]} skipped")
            number_skipped += 1
        else:
            if condition not in exps:
                exps[condition] = []
            # else:
            #     print(f"{line[ID]} already exists")

            exps[condition].append(response)
            if file_name is not None:
                fp.write("\t".join(line)+"\n")

    file.close()
    if file_name is not None:
        fp.close()
    
    print(f"Skipped {number_skipped} experiments")

    return exps

def write_exps(exps, file_name):

    fp = open(file_name, "w")

    for condition in exps:
        fp.write(f"Condition\n")
        fp.write(f"{condition}\n")
        fp.write(f"Response\n")
        for response in exps[condition]:
            fp.write(f"{response}\t")
        fp.write(f"\nWeight:{len(exps[condition])}\n\n")

    fp.close()

exps = convert_exps(ABA_A_data, constants, "ABA_A_gitsbe.tsv")
write_exps(exps, "ABA_A_training_data.tab")

print(len(exps))

Skipped 76 experiments
207


In [13]:
# Certain node names are not allowed by Gitsbe. Change them.

def inplace_change(filename, old_string, new_string):
    # Safely read the input filename using 'with'
    with open(filename) as f:
        s = f.read()
        if old_string not in s:
            print('"{old_string}" not found in {filename}.'.format(**locals()))
            return

    # Safely write the changed content, if found in the file
    with open(filename, 'w') as f:
        print('Changing "{old_string}" to "{new_string}" in {filename}'.format(**locals()))
        s = s.replace(old_string, new_string)
        f.write(s)

for file in ["ABA_A_network.sif", "ABA_A_training_data.tab", "ABA_A_gitsbe.tsv"]:
    inplace_change(file, "8-nitro-cGMP", "nitro_cGMP")
    inplace_change(file, "V-PPase", "V_PPase")
    inplace_change(file, "V-ATPase", "V_ATPase")


Changing "8-nitro-cGMP" to "nitro_cGMP" in ABA_A_network.sif
Changing "V-PPase" to "V_PPase" in ABA_A_network.sif
Changing "V-ATPase" to "V_ATPase" in ABA_A_network.sif
Changing "8-nitro-cGMP" to "nitro_cGMP" in ABA_A_training_data.tab
Changing "V-PPase" to "V_PPase" in ABA_A_training_data.tab
Changing "V-ATPase" to "V_ATPase" in ABA_A_training_data.tab
Changing "8-nitro-cGMP" to "nitro_cGMP" in ABA_A_gitsbe.tsv
Changing "V-PPase" to "V_PPase" in ABA_A_gitsbe.tsv
Changing "V-ATPase" to "V_ATPase" in ABA_A_gitsbe.tsv
